In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper

In [ ]:
import pandas as pd
# Load the dataset
file_path = "E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/test_data_shap_metrics.csv"  # Replace with the actual file path
df = pd.read_csv(file_path)

# Convert date columns to datetime format
df['Start_Date'] = pd.to_datetime(df['Start_Date'])
df['End_Date'] = pd.to_datetime(df['End_Date'])

# Pivot the table to transform it into a time series format with Start_Date and End_Date
df_pivot = df.pivot_table(
    index=['Start_Date', 'End_Date'], 
    columns='Stock', 
    values=[col for col in df.columns if col not in ['Stock', 'Start_Date', 'End_Date', 'Phase']]
)

# Flatten the multi-index columns
df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]

# Sort the dataframe by Start_Date and End_Date
df_pivot.sort_index(inplace=True)

# Save the transformed data to a new CSV file
df_pivot.to_csv("transformed_shap_timeseries.csv")

print("Transformation complete. Data saved as 'transformed_shap_timeseries.csv'.")


In [ ]:
import pandas as pd
import numpy as np

# Load the transformed SHAP dataset
shap_file_path = "E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/transformed_shap_timeseries.csv"
df_shap = pd.read_csv(shap_file_path, index_col=[0, 1])

# Load the ETF daily returns dataset
returns_file_path = "E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/daily_returns.csv"
df_returns = pd.read_csv(returns_file_path, parse_dates=["Date"], index_col="Date")

# Ensure ETF tickers are correctly identified
etfs = list(set([col.split("_")[-1] for col in df_shap.columns if "mean_return" not in col and "volatility" not in col]))

# Initialize columns for mean return and volatility
for etf in etfs:
    df_shap[f"mean_return_{etf}"] = np.nan
    df_shap[f"volatility_{etf}"] = np.nan

# Compute mean return and volatility for each ETF based on Start_Date and End_Date
for (start_date, end_date) in df_shap.index[[0]]:

    # Convert to datetime format
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get mean return over the period
    period_returns = df_returns.loc[start_date:end_date]
    mean_returns = period_returns.mean()
    period_returns_df = pd.DataFrame(period_returns)
    # Get volatility using past 60 days from end_date
    past_60_returns = df_returns.loc[:end_date].iloc[-60:]
    volatilities = past_60_returns.std()

    # Assign computed values to df_shap
    for etf in etfs:
        if etf in mean_returns and etf in volatilities:
            df_shap.at[(start_date, end_date), f"mean_return_{etf}"] = mean_returns[etf]
            df_shap.at[(start_date, end_date), f"volatility_{etf}"] = volatilities[etf]

# Save the updated dataset
updated_shap_file_path = "transformed_shap_timeseries_updated_test.csv"
df_shap.to_csv(updated_shap_file_path)

print(f"Updated dataset saved as {updated_shap_file_path}")


In [ ]:

file_path = "transformed_shap_timeseries_updated.csv"
df_shap = pd.read_csv(file_path, index_col=[0, 1])
# Sort columns first by feature type, then by ETF
feature_order = sorted(df_shap.columns, key=lambda x: (x.split('_')[0], x.split('_')[-1]))
df_shap = df_shap[feature_order]


In [ ]:
df_shap.to_csv("transformed_shap_timeseries_updated2.csv")

In [ ]:
# Load the daily returns dataset
daily_returns_path = "daily_returns.csv"
df_returns = pd.read_csv(daily_returns_path, parse_dates=["Date"], index_col="Date")

# Sort daily return columns in the same order as feature_order
sorted_etfs =  [col.replace("mean_return_", "") for col in feature_order if "mean_return" in col]
df_returns = df_returns[sorted_etfs]
df_returns.to_csv("daily_returns2.csv")

In [ ]:
etfs

In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import pandas as pd
import numpy as np
import gymnasium as gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import os
file_path = "E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/transformed_shap_timeseries_updated_sort.csv"
df_shap = pd.read_csv(file_path, index_col=[0, 1])

# Extract ETFs
etfs = list(set([col.split("_")[-1] for col in df_shap.columns if "mean_return" in col]))
print(f'etfs={etfs}')

global num_etfs
num_etfs = len(etfs)
print(f'num_etfs={num_etfs}')
daily_returns_path = "daily_returns_sort.csv"
df_returns = pd.read_csv(daily_returns_path, parse_dates=["Date"], index_col="Date")
# Train agent with rolling 78-sample window
window_size = 78
num_samples = len(df_shap)
out_of_sample_weights = []

for start in range(0, num_samples - window_size, window_size):
    end = start + window_size
    train_data = df_shap.iloc[start:end]
    env = make_vec_env(lambda: PortfolioTradingEnv(train_data, df_returns,etfs), n_envs=1)
    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=10000)
    
    # Save trained model
    model_path = f"ppo_portfolio_model_{start}_{end}.zip"
    model.save(model_path)
    print(f"Model saved: {model_path}")
    
    # Generate out-of-sample weights
    test_data = df_shap.iloc[end:end+1]
    obs = test_data.values
    weights_list = []
    for _ in range(100):
        action, _ = model.predict(obs, deterministic=True)
        weights_list.append(action)
    avg_weights = np.mean(weights_list, axis=0)
    out_of_sample_weights.append(avg_weights)

# Save the results
out_of_sample_df = pd.DataFrame(out_of_sample_weights, columns=etfs)
out_of_sample_df.to_csv("out_of_sample_weights.csv")
print("Training complete. Models and out-of-sample weights saved.")



In [1]:
import pandas as pd
import numpy as np
import gymnasium as gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import os

# Load the transformed SHAP dataset
file_path = "transformed_shap_timeseries_updated_sort.csv"
df_shap = pd.read_csv(file_path, index_col=[0, 1])

# Load the daily returns dataset
daily_returns_path = "daily_returns_sort.csv"
df_returns = pd.read_csv(daily_returns_path, parse_dates=["Date"], index_col="Date")

# Extract ETFs
etfs = list(set([col.split("_")[-1] for col in df_shap.columns if "mean_return" in col]))
global num_etfs
num_etfs = len(etfs)

class PortfolioTradingEnv(gym.Env):
    def __init__(self, data, returns, initial_balance=1000, etfs=None):
        super(PortfolioTradingEnv, self).__init__()
        self.data = data
        self.returns = returns
        self.initial_balance = initial_balance
        self.current_step = 0
        self.etfs = etfs if etfs else sorted_etfs
        self.num_etfs = len(self.etfs)
        self.etfs = etfs if etfs else []
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)
        self.action_space = spaces.Box(low=0, high=1.0, shape=(12,), dtype=np.float32)
        self.portfolio_value = self.initial_balance
        self.weights = np.ones(self.num_etfs) / self.num_etfs
    def seed(self, seed=None):
        np.random.seed(seed)
    def __init__(self, data, returns, initial_balance=1000):
        super(PortfolioTradingEnv, self).__init__()
        self.data = data
        self.returns = returns
        self.initial_balance = initial_balance
        self.current_step = 0
        self.etfs = etfs if etfs else sorted_etfs
        self.num_etfs = len(self.etfs)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)
        self.action_space = spaces.Box(low=np.array([0.0] * 12, dtype=np.float32), high=np.array([1.0] * 12, dtype=np.float32), dtype=np.float32)
        self.portfolio_value = self.initial_balance
        self.weights = np.ones(self.num_etfs) / self.num_etfs

    def reset(self):
        self.current_step = 0
        self.portfolio_value = self.initial_balance
        self.weights = np.ones(self.num_etfs) / self.num_etfs
        return self.data.iloc[self.current_step].values

    def step(self, action):
        action = np.array(action, dtype=np.float32)
        action = np.clip(action, 0, 1)
        action /= np.sum(action)
        self.weights = action

        start_date, end_date = self.data.index[self.current_step]
        period_returns = self.returns.loc[start_date:end_date, self.etfs].values
        portfolio_values = [self.portfolio_value]
        for daily_return in period_returns:
            self.portfolio_value *= (1 + np.dot(daily_return, self.weights))
            self.weights *= (1 + daily_return)
            self.weights /= np.sum(self.weights)  # Normalize weights after drift
            portfolio_values.append(self.portfolio_value)
        cumulative_return = (portfolio_values[-1] / portfolio_values[0]) - 1
        

        self.portfolio_value *= (1 + cumulative_return)
        reward = np.log(self.portfolio_value / self.initial_balance)

        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        next_state = self.data.iloc[self.current_step].values if not done else np.zeros_like(self.data.iloc[0].values)
        return next_state, reward, done, {}

    def render(self):
        print(f"Step: {self.current_step}, Portfolio Value: {self.portfolio_value:.2f}")

# Train agent with rolling 78-sample window
window_size = 78
num_samples = len(df_shap)
out_of_sample_weights = []

for start in range(0, num_samples - window_size, window_size):
    end = start + window_size
    train_data = df_shap.iloc[start:end]
    env = make_vec_env(lambda: PortfolioTradingEnv(train_data, df_returns), n_envs=1)
    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=10000)
    
    # Save trained model
    model_path = f"ppo_portfolio_model_{start}_{end}.zip"
    model.save(model_path)
    print(f"Model saved: {model_path}")
    
    # Generate out-of-sample weights
    test_data = df_shap.iloc[end:end+1]
    obs = test_data.values
    weights_list = []
    for _ in range(100):
        action, _ = model.predict(obs, deterministic=True)
        weights_list.append(action)
    avg_weights = np.mean(weights_list, axis=0)
    out_of_sample_weights.append(avg_weights)

# Save the results
out_of_sample_df = pd.DataFrame(out_of_sample_weights, columns=etfs)
out_of_sample_df.to_csv("out_of_sample_weights.csv")
print("Training complete. Models and out-of-sample weights saved.")


Using cpu device


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.box.Box'>, <class 'gymnasium.spaces.discrete.Discrete'>, <class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>, <class 'gymnasium.spaces.multi_binary.MultiBinary'>) as action spaces but Box(0.0, 1.0, (12,), float32) was provided